In [ ]:
%pip install azure-search-documents==11.6.0b9 azure-identity python-dotenv pandas jinja2 --quiet

In [15]:
import os
from azure.core.credentials import AzureKeyCredential
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

load_dotenv(override=True) # take environment variables from .env.

endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
index_name = os.environ["AZURE_SEARCH_INDEX"]
credential = AzureKeyCredential(os.getenv("AZURE_SEARCH_ADMIN_KEY")) if os.getenv("AZURE_SEARCH_ADMIN_KEY") else DefaultAzureCredential()

## Setup API Client

In [16]:
from azure.search.documents import SearchClient
import pandas as pd

search_client = SearchClient(endpoint, index_name, credential)

def display_results(results):
    df = pd.json_normalize(list(results)).dropna(axis=1, how='all')
    df["chunk"] = df["chunk"].apply(lambda c: c[:300] + '...' if len(c) > 300 else c)
    first_cols = ['title', 'chunk', '@search.score']
    df = df[first_cols + [col for col in df.columns if col not in first_cols]]

    df = df.style.set_properties(**{
        'max-width': '500px',
        'text-align': 'left',
        'white-space': 'normal',
        'word-wrap': 'break-word'
    }).hide(axis="index")


    return df


## Keyword Search

In [ ]:
results = search_client.search(search_text="What is Contoso", top=5, select=["title", "chunk"])

display_results(results)


## Vector Search

In [ ]:
from azure.search.documents.models import VectorizableTextQuery

results = search_client.search(vector_queries=[VectorizableTextQuery(text="What is Contoso", k_nearest_neighbors=50, fields="text_vector")], top=5, select=["title", "chunk"])

display_results(results)

## Hybrid Search

In [ ]:
results = search_client.search(
    search_text="What is Contoso",
    vector_queries=[VectorizableTextQuery(text="What is Contoso", k_nearest_neighbors=50, fields="text_vector")],
    top=5,
    select=["title", "chunk"]
)

display_results(results)

## Hybrid Search + Semantic Ranker

In [ ]:
results = search_client.search(
    search_text="What is Contoso",
    vector_queries=[VectorizableTextQuery(text="What is Contoso", k_nearest_neighbors=50, fields="text_vector")],
    top=5,
    select=["title", "chunk"],
    query_type="semantic",
    semantic_configuration_name="ragtime2-semantic-configuration"
)

display_results(results)

## Hybrid Search + Semantic Ranker + Query Rewriting

In [ ]:
results = search_client.search(
    search_text="What is Contoso",
    vector_queries=[VectorizableTextQuery(text="What is Contoso", k_nearest_neighbors=50, fields="text_vector")],
    top=5,
    select=["title", "chunk"],
    query_type="semantic",
    semantic_configuration_name="ragtime2-semantic-configuration",
    query_rewrites="generative",
    query_language="en"
)

display_results(results)